In [ ]:
import pandas as pd

def getFileForSTTest(predictions_file_t5, predictions_file_baseline, target_file, dataset='javadoc'):


    target = open(target_file,'r')
    target_list =  target.readlines()
    
    baseline_predictions = []
    t5_predictions = []
    
    with open(predictions_file_t5) as fread:
        t5_predictions = fread.readlines()
        
    with open(predictions_file_baseline) as fread:
        baseline_predictions = fread.readlines()
    
    assert(len(target_list) == len(baseline_predictions) == len(t5_predictions))
    
    
    method_id = []
    prediction_size = [] 
    is_perfect_t5 = []
    is_perfect_baseline = []
    
    
    for i in range(1,16):

        for (id_method,( ref, pred_t5, pred_baseline )) in enumerate(zip(target_list,t5_predictions,baseline_predictions)):

            pred_t5 = pred_t5.strip().lower().split()
            pred_t5 = pred_t5[0:i]
            
            pred_baseline = pred_baseline.strip().lower().split()
            pred_baseline = pred_baseline[0:i]

            ref = ref.strip().lower().split()
            ref = ref[0:i]

            if len(ref)>=i: #and len(pred)>=i:
                
                cmp_ref = ''.join(ref)
             
                if ''.join(pred_t5) == cmp_ref:
                    
                    if ''.join(pred_baseline) == cmp_ref:
                        flag_t5 = True
                        flag_baseline = True
                    
                    else:
                        flag_t5 = True
                        flag_baseline = False
             
                elif ''.join(pred_baseline) == cmp_ref:
                    flag_t5 = False
                    flag_baseline = True
                
                else:
                    flag_t5 = False
                    flag_baseline = False
                    
            idx = '{}_{}_{}'.format(dataset, id_method, i)
            method_id.append(idx)
            prediction_size.append(i)
            is_perfect_t5.append(flag_t5)
            is_perfect_baseline.append(flag_baseline)

 
    st_test = pd.DataFrame(zip(method_id, prediction_size, is_perfect_t5, is_perfect_baseline),
                           columns=['method_id','prediction_size','is_perfect_T5','is_perfect_baseline'])    
    return st_test

In [ ]:
####### EXTRACT FILE FOR STATISTICAL TEST #######
import pandas as pd

path_to_predictions_jdoc_t5 = ''
path_to_predictions_jdoc_baseline = ''
path_to_targets_jdoc = ''


data = getFileForSTTest(path_to_predictions_jdoc_t5, path_to_predictions_jdoc_baseline, path_to_targets_jdoc, dataset='javadoc')
data.to_csv('result_comparison_javadoc.csv',index=False)


path_to_predictions_inside_t5 = ''
path_to_predictions_inside_baseline = ''
path_to_targets_inside = ''


data = getFileForSTTest(path_to_predictions_inside_t5, path_to_predictions_inside_baseline, path_to_targets_inside, dataset='inside')
data.to_csv('result_comparison_inside.csv',index=False)

In [ ]:
df_javadoc = pd.read_csv('result_comparison_javadoc.csv')
df_inside = pd.read_csv('result_comparison_inside.csv')
df_overall = pd.concat([df_inside, df_javadoc])

assert(len(df_overall) == len(df_javadoc) + len(df_inside))

df_overall.to_csv('result_comparison_overall.csv')

In [ ]:
!/Library/Frameworks/R.framework/Versions/4.0/Resources/bin/Rscript statistical_test.r